### Export TSOSI data file


In [ ]:
from pathlib import Path
import sys
import os
import django

# Add the parent directory to the system path and setup django
BASE_DIR = str(Path(os.getcwd()).resolve().parent.parent)

if BASE_DIR not in sys.path:
    sys.path.append(BASE_DIR)

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "backend_site.settings")

django.setup()

from tsosi.data.preparation.doaj.publishers_202X import (
    get_config as doaj_publisher_config,
)
from tsosi.data.preparation.doaj.libraries_2024_now import (
    get_config as doaj_library_2024_config,
    get_fixture_config as doaj_library_fixture_2024_config,
)
from tsosi.data.preparation.doaj.libraries_2023 import (
    get_config as doaj_library_2023_config,
)
from tsosi.data.preparation.doaj.libraries_2021_2022 import (
    get_config as doaj_library_2021_2022_config,
)

year = 2024
file_path = (
    Path.home()
    / "Nextcloud/TSOSI_data/doaj/4_prepared/2025-01-07-DOAJ_Publisher_Report_2024_prepared.xlsx"
)
file_path = f"{BASE_DIR}/tsosi/data/fixtures/doaj_example.json"
sheet_name = "Sheet1"
config = doaj_library_fixture_2024_config(year, str(file_path))
config.get_data()
# config = doaj_library_2023_config(file_path, sheet_name)
config.generate_data_file()

### DEV


In [1]:
from pathlib import Path
import sys
import os
import django
import pandas as pd
from asgiref.sync import sync_to_async

# Add the parent directory to the system path and setup django
BASE_DIR = str(Path(os.getcwd()).resolve().parent.parent)

if BASE_DIR not in sys.path:
    sys.path.append(BASE_DIR)

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "backend_site.settings")

django.setup()

from tsosi.data.db_utils import (
    date_extremas_from_queryset,
    DateExtremas,
    bulk_create_from_df,
    bulk_update_from_df,
)
from tsosi.models import Transfert, Entity, InfrastructureDetails
from django.db.models import Q, Count, Max


@sync_to_async
def run():
    # Min & max transfert dates per recipient
    dates = date_extremas_from_queryset(
        Transfert.objects.all(), ["date_clc"], groupby=["recipient_id"]
    )
    dates: dict[str, DateExtremas] = {
        d["recipient_id"]: d["_extremas"] for d in dates
    }

    # Ratio of hidden amounts among all amounts per recipient
    hidden_transferts = Count(
        "transfert_as_recipient",
        filter=Q(transfert_as_recipient__hide_amount=True),
    )
    total_transferts = Count("transfert_as_recipient")
    date_source_max = Max(
        "transfert_as_recipient__data_load_source__date_data_obtained"
    )

    entities = (
        Entity.objects.filter(is_recipient=True)
        .annotate(hidden_transferts=hidden_transferts)
        .annotate(total_transferts=total_transferts)
        .annotate(date_last_update=date_source_max)
        .values(
            "id",
            "hidden_transferts",
            "total_transferts",
            "date_last_update",
            "infrastructure_details",
        )
    )
    data = pd.DataFrame.from_records(entities)
    if data.empty:
        return data, dates

    data["hidden_ratio"] = data["hidden_transferts"] / data["total_transferts"]

    data["dates"] = data["id"].map(dates)
    data["date_data_start"] = data["dates"].apply(
        lambda x: x.min if not pd.isna(x) else x
    )
    data["date_data_end"] = data["dates"].apply(
        lambda x: x.max if not pd.isna(x) else x
    )

    data["date_data_update"] = data["date_last_update"]

    no_details_mask = data["infrastructure_details"].isna()
    details_to_create = data[no_details_mask].copy()
    if not details_to_create.empty:
        details_to_create["entity_id"] = details_to_create["id"]
        bulk_create_from_df(
            InfrastructureDetails,
            details_to_create,
            [
                "entity_id",
                "hidden_ratio",
                "data_data_start",
                "date_data_end",
                "date_data_update",
            ],
        )

    details_to_update = data[~no_details_mask].copy()
    if not details_to_update.empty:
        details_to_update["id"] = details_to_update["infrastructure_details"]
        bulk_update_from_df(
            InfrastructureDetails,
            details_to_update,
            [
                "id",
                "hidden_ratio",
                "date_data_start",
                "date_data_end",
                "date_data_update",
            ],
        )

    return data, dates


res = await run()

/home/guillaume-alzieu/Dev/TSOSI/tsosi-app/backend/.venv/lib/python3.12/site-packages/django/db/models/fields/__init__.py:1606: RuntimeWarning: DateTimeField InfrastructureDetails.date_data_update received a naive datetime (2025-02-11 00:00:00) while time zone support is active.
  warnings.warn(


In [14]:
data, dates = res
data

,id,infrastructure_details,hidden_transferts,total_transferts,date_last_update,hidden_ratio,dates,date_data_start,date_data_end,date_data_update
0,b84cbff0-f2d0-46db-bc45-73caf5a1e33c,2,23,23,2025-02-11,1.0,"DateExtremas(min=datetime.date(2021, 5, 11), m...",2021-05-11,2025-01-01,2025-02-11


In [1]:
from fontTools.ttLib.woff2 import compress
from pathlib import Path
import re

font_directory = Path.home() / "Documents/TSOSI/Graphisme/Mont/Mont"
otf_files = font_directory.glob("*.otf")
save_directory = Path.home() / "Documents/TSOSI/Fonts/Mont"

for file in otf_files:
    file_path_usefull = re.match(r".*(Mont[^\.]+)", file.name)
    save_path = (
        str(save_directory / file_path_usefull[1]).replace(" ", "_") + ".woff2"
    )
    compress(str(file), str(save_path))

file: /home/guillaume-alzieu/Documents/TSOSI/Graphisme/Mont/Mont/Fontfabric - Mont Bold Italic.otf
Save path: /home/guillaume-alzieu/Documents/TSOSI/Fonts/Mont/Mont_Bold_Italic.woff2
file: /home/guillaume-alzieu/Documents/TSOSI/Graphisme/Mont/Mont/Fontfabric - Mont Hairline Italic.otf
Save path: /home/guillaume-alzieu/Documents/TSOSI/Fonts/Mont/Mont_Hairline_Italic.woff2
file: /home/guillaume-alzieu/Documents/TSOSI/Graphisme/Mont/Mont/Fontfabric - Mont Regular Italic.otf
Save path: /home/guillaume-alzieu/Documents/TSOSI/Fonts/Mont/Mont_Regular_Italic.woff2
file: /home/guillaume-alzieu/Documents/TSOSI/Graphisme/Mont/Mont/Fontfabric - Mont Black.otf
Save path: /home/guillaume-alzieu/Documents/TSOSI/Fonts/Mont/Mont_Black.woff2
file: /home/guillaume-alzieu/Documents/TSOSI/Graphisme/Mont/Mont/Fontfabric - Mont Regular.otf
Save path: /home/guillaume-alzieu/Documents/TSOSI/Fonts/Mont/Mont_Regular.woff2
file: /home/guillaume-alzieu/Documents/TSOSI/Graphisme/Mont/Mont/Fontfabric - Mont Thin.otf